In [ ]:
import requests
import json

# Testa performance
with open("batch_records.json", "r") as f:
    records = json.load(f)

resp = requests.post("http://localhost:8001/v1/performance", json=records)
print(resp.json())

# Testa aderência
resp = requests.post("http://localhost:8001/v1/aderencia", json={"dataset_path": "../datasets/credit_01/train.gz"})
print(resp.json())

resp = requests.post("http://localhost:8001/v1/aderencia", json={"dataset_path": "../datasets/credit_01/oot.gz"})
print(resp.json())
